In [2]:
# 20201120 1603958 8804899 975601
import pandas as pd
orderLog = pd.read_csv(r'\\192.168.10.34\random_backup\Kevin_zhenyu\orderLog\equityTradeLogs\speedCompare_20201120.csv')
SH = pd.read_csv(r'\\192.168.10.34\random_backup\Kevin_zhenyu\rawData\logs_20201120_zs_92_01_day_data\mdLog_SH_20201120_0902.csv')

C:\Users\win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [73]:
SH['ID'] = SH['StockID'] + 1000000
SH['time'] = SH.time.str.slice(0, 2) + SH.time.str.slice(3, 5) + SH.time.str.slice(6, 8) + '000'
SH['time'] = SH['time'].astype('int64')
SH['time'] = SH.groupby(['ID'])['time'].cummax()
SH['max_cum_volume'] = SH.groupby(['StockID'])['cum_volume'].cummax()
indexData = SH[SH['StockID'].isin([300, 852, 905])][['ID', 'sequenceNo', 'close']].reset_index(drop=True)
SH = SH[SH['StockID'] >= 600000]
SH = SH[(SH['cum_volume'] > 0) & (SH['time'] >= 93000000) &\
                    (SH['cum_volume'] == SH['max_cum_volume'])]
SH = SH[['ID', 'clockAtArrival', 'sequenceNo', 'time', 'cum_volume', 'bid1p', 'ask1p', 'bid1q', 'ask1q', 'bid5q', 'ask5q']]
SH

,ID,clockAtArrival,sequenceNo,time,cum_volume,bid1p,ask1p,bid1q,ask1q,bid5q,ask5q
286604,1600060,1605835808382649,1975291,93000000,163400,11.58,11.59,4000,5900,700,1000
286605,1600196,1605835808382657,1975292,93000000,223500,52.60,52.61,19000,6100,600,3500
286606,1600213,1605835808382661,1975293,93000000,695900,7.90,7.93,51500,3700,321600,5400
286607,1600330,1605835808382666,1975294,93000000,26692,9.38,9.40,3408,8100,2400,82200
286608,1600418,1605835808382671,1975295,93000000,2011000,9.09,9.10,41100,8900,66300,15400
...,...,...,...,...,...,...,...,...,...,...,...
13324795,1603596,1605856359944203,131302507,151231000,1906603,40.57,40.60,800,11200,8600,14800
13324796,1600968,1605856359951007,131302508,151231000,21057700,2.44,2.45,3447200,226900,466000,1973793
13324797,1688181,1605856359956081,131302509,151231000,398719,62.82,62.83,612,2327,508,300
13324798,1603290,1605856359956095,131302510,151231000,2788602,213.89,213.90,400,3400,200,100


In [6]:
import pymongo
import pandas as pd
import numpy as np
import pickle
import datetime
import time
import gzip
import lzma
import pytz

def DB(host, db_name, user, passwd):
    auth_db = db_name if user not in ('admin', 'root') else 'admin'
    url = 'mongodb://%s:%s@%s/?authSource=%s' % (user, passwd, host, auth_db)
    client = pymongo.MongoClient(url, maxPoolSize=None)
    db = client[db_name]
    return db

def read_memb_daily(db, name, start_date=None, end_date=None, skey=None, index_id=None, interval=None, col=None, return_sdi=True):
    collection = db[name]
    # Build projection
    prj = {'_id': 0}
    if col is not None:
        if return_sdi:
            col = ['skey', 'date', 'interval'] + col
        for col_name in col:
            prj[col_name] = 1

    # Build query
    query = {}
    if skey is not None:
        query['skey'] = {'$in': skey}
    if index_id is not None:
        query['index_id'] = {'$in': index_id}
    if interval is not None:
        query['interval'] = {'$in': interval}
    if start_date is not None:
        if end_date is not None:
            query['date'] = {'$gte': start_date, '$lte': end_date}
        else:
            query['date'] = {'$gte': start_date}
    elif end_date is not None:
        query['date'] = {'$lte': end_date}

    # Load data
    cur = collection.find(query, prj)
    df = pd.DataFrame.from_records(cur)
    if df.empty:
        df = pd.DataFrame()
    else:
        df = df.sort_values(by=['date', 'index_id', 'skey'])
    return df    

def read_beta_daily(db, name, start_date=None, end_date=None, skey=None, interval=None, col=None, return_sdi=True): 
    collection = db[name] 
    # Build projection 
    prj = {'_id': 0} 
    if col is not None: 
        if return_sdi: 
            col = ['skey', 'date'] + col 
        for col_name in col: 
            prj[col_name] = 1 
 
    # Build query 
    query = {} 
    if skey is not None: 
        query['skey'] = {'$in': skey} 
    if interval is not None: 
        query['interval'] = {'$in': interval} 
    if start_date is not None: 
        if end_date is not None: 
            query['date'] = {'$gte': start_date, '$lte': end_date} 
        else: 
            query['date'] = {'$gte': start_date} 
    elif end_date is not None: 
        query['date'] = {'$lte': end_date} 
 
    # Load data 
    cur = collection.find(query, prj) 
    df = pd.DataFrame.from_records(cur) 
    if df.empty: 
        df = pd.DataFrame() 
    else: 
        df = df.sort_values(by=['date','skey']) 
    return df  

def build_query(start_date=None, end_date=None, index_id=None):
    query = {}
    def parse_date(x):
        if type(x) == int:
            return x
        elif type(x) == str:
            if len(x) != 8:
                raise Exception("date must be YYYYMMDD format")
            return int(x)
        elif type(x) == datetime.datetime or type(x) == datetime.date:
            return x.strftime("%Y%m%d").astype(int)
        else:
            raise Exception("invalid date type: " + str(type(x)))
    if start_date is not None or end_date is not None:
        query['date'] = {}
        if start_date is not None:
            query['date']['$gte'] = parse_date(start_date)
        if end_date is not None:
            query['date']['$lte'] = parse_date(end_date)
    def parse_symbol(x):
        if type(x) == int:
            return x
        else:
            return int(x)
    if index_id:
        if type(index_id) == list or type(index_id) == tuple:
            query['index_id'] = {'$in': [parse_symbol(x) for x in index_id]}
        else:
            query['index_id'] = parse_symbol(index_id)
    return query

def write_memb_data(db, name, df):
    collection = db[name]
    df1 = []
    for symbol in df['index_id'].unique():
        if symbol in collection.distinct('index_id'):
            symbol = int(symbol)
            m_ax = pd.DataFrame.from_records(collection.find({'index_id':{'$in':[symbol]}}).sort([('date',-1)]).skip(0).limit(1))['date'].values[0]
            df2 = df[(df['index_id'] == symbol) & (df['date'] > m_ax)]
            print(df2)
            df1 += [df2]
        else:
            print(symbol)
            df2 = df[(df['index_id'] == symbol)]
            print(df2)
            df1 += [df2]
    df1 = pd.concat(df1).reset_index(drop=True)
    df1 = df1.to_dict('records')
    collection.insert_many(df1) 

def delete_memb_data(db, name, start_date=None, end_date=None, index_id=None):
    collection = db[name]
    query = build_query(start_date, end_date, index_id)
    if not query:
        print('cannot delete the whole table')
        return None
    collection.delete_many(query)  


database_name = 'com_md_eq_cn'
user = "zhenyuy"
password = "bnONBrzSMGoE"

pd.set_option('max_columns', 200)
db1 = DB("192.168.10.178", database_name, user, password)
indexCat = read_memb_daily(db1, 'index_memb', 20201120, 20201120)
indexCat[indexCat['skey'] == 1603958]

,date,weight,skey,index_id,index_name
2377,20201120,0.016208,1603958,1000985,CSIRest


In [74]:
SH = SH[SH['ID'].isin([1603958, 1000852])]
SH = pd.concat([SH, indexData[indexData['ID'] == 1000852]])
SH

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,ID,ask1p,ask1q,ask5q,bid1p,bid1q,bid5q,clockAtArrival,close,cum_volume,sequenceNo,time
287129,1603958,8.06,11599.0,3100.0,8.05,8600.0,2100.0,1.605836e+15,NaN,23700.0,1999915,93000000.0
288159,1603958,8.06,8399.0,3100.0,8.05,8600.0,2100.0,1.605836e+15,NaN,26900.0,2039987,93000000.0
290517,1603958,8.12,27700.0,7100.0,8.06,6701.0,2100.0,1.605836e+15,NaN,81699.0,2436140,93002000.0
292019,1603958,8.12,27700.0,7100.0,8.06,5501.0,2100.0,1.605836e+15,NaN,82899.0,2489528,93003000.0
294339,1603958,8.11,6000.0,5000.0,8.06,5301.0,1500.0,1.605836e+15,NaN,91799.0,2877365,93006000.0
...,...,...,...,...,...,...,...,...,...,...,...,...
12700,1000852,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6766.0605,NaN,131284736,NaN
12704,1000852,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6766.0605,NaN,131291978,NaN
12708,1000852,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6766.0605,NaN,131294338,NaN
12709,1000852,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6766.0605,NaN,131294617,NaN


In [17]:
SH1 = pd.read_csv(r'\\192.168.10.34\random_backup\Kevin_zhenyu\rawData\logs_20201120_zs_92_01_day_data\mdLog_SH_20201120_0902.csv')

In [21]:
SH1[(SH1['StockID'] == 852) & (SH1['sequenceNo'] > 2009997)]

,ms,clock,threadId,clockAtArrival,sequenceNo,source,StockID,exchange,time,cum_volume,cum_amount,close,__origTickSeq,bid1p,bid2p,bid3p,bid4p,bid5p,bid1q,bid2q,bid3q,bid4q,bid5q,ask1p,ask2p,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice,numTrades
292780,09:30:12.587714,1605835812589112,20919,1605835812588767,2586824,5,852,SH,09:30:04.000,1148897,1.283474e+09,6712.4320,1015019295972954279,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,6713.6481,0
297864,09:30:17.546714,1605835817547906,20919,1605835817547870,3126584,5,852,SH,09:30:09.000,1975188,2.171412e+09,6710.3909,72353251007896961,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,6713.6481,0
304305,09:30:22.513715,1605835822515280,20919,1605835822515256,3304855,5,852,SH,09:30:14.000,2171887,2.384412e+09,6708.4409,72353251008106490,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,6713.6481,0
310062,09:30:27.416715,1605835827417652,20919,1605835827417431,3452311,5,852,SH,09:30:19.000,2171887,2.384412e+09,6708.4409,72353251008280435,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,6713.6481,0
310113,09:30:27.492715,1605835827494304,20919,1605835827494276,3455666,5,852,SH,09:30:19.000,2286384,2.521475e+09,6707.0773,1015019295973935345,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,6713.6481,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13316256,15:11:02.934642,1605856262936396,20919,1605856262936361,131284736,5,852,SH,15:10:54.000,133167459,1.525199e+11,6766.0605,72353251209820572,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,6713.6481,0
13318882,15:11:32.941644,1605856292942684,20919,1605856292942651,131291978,5,852,SH,15:11:24.000,133167459,1.525199e+11,6766.0605,72353251209918323,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,6713.6481,0
13321242,15:11:59.583645,1605856319584932,20919,1605856319584896,131294338,5,852,SH,15:11:50.000,133167459,1.525199e+11,6766.0605,1015019296175603125,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,6713.6481,0
13321521,15:12:02.946645,1605856322948173,20919,1605856322948148,131294617,5,852,SH,15:11:54.000,133167459,1.525199e+11,6766.0605,72353251209956290,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,6713.6481,0


In [16]:
SH[SH['ID'] == 1000852].head(100)

,ID,ask1p,ask1q,ask5q,bid1p,bid1q,bid5q,clockAtArrival,close,cum_volume,sequenceNo,time
1,1000852,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6718.4582,NaN,68,NaN
3,1000852,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6718.4582,NaN,5076,NaN
5,1000852,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6718.4582,NaN,5158,NaN
8,1000852,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6718.4582,NaN,5545,NaN
11,1000852,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6718.4582,NaN,5921,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
278,1000852,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6718.4582,NaN,66851,NaN
280,1000852,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6718.4582,NaN,67234,NaN
282,1000852,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6718.4582,NaN,67330,NaN
284,1000852,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6718.4582,NaN,67461,NaN


In [75]:
SH = SH.sort_values(by='sequenceNo')
SH['indexClose'] = np.where(SH['ID'] == 1000852, SH['close'], np.nan)
SH['indexClose'] = SH['indexClose'].ffill()
SH[SH['sequenceNo'] >= 2009997].head(50)

,ID,ask1p,ask1q,ask5q,bid1p,bid1q,bid5q,clockAtArrival,close,cum_volume,sequenceNo,time,indexClose
288159,1603958,8.06,8399.0,3100.0,8.05,8600.0,2100.0,1.605836e+15,NaN,26900.0,2039987,93000000.0,6713.6481
290517,1603958,8.12,27700.0,7100.0,8.06,6701.0,2100.0,1.605836e+15,NaN,81699.0,2436140,93002000.0,6713.6481
292019,1603958,8.12,27700.0,7100.0,8.06,5501.0,2100.0,1.605836e+15,NaN,82899.0,2489528,93003000.0,6713.6481
1055,1000852,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6712.4320,NaN,2586824,NaN,6712.4320
294339,1603958,8.11,6000.0,5000.0,8.06,5301.0,1500.0,1.605836e+15,NaN,91799.0,2877365,93006000.0,6712.4320
295681,1603958,8.08,1000.0,13500.0,8.06,1901.0,1500.0,1.605836e+15,NaN,95299.0,2922269,93006000.0,6712.4320
1058,1000852,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6710.3909,NaN,3126584,NaN,6710.3909
298132,1603958,8.05,2299.0,7900.0,8.03,3500.0,500.0,1.605836e+15,NaN,100000.0,3128166,93009000.0,6710.3909
299189,1603958,8.05,2299.0,7900.0,8.03,3500.0,500.0,1.605836e+15,NaN,100000.0,3135918,93009000.0,6710.3909
301992,1603958,8.05,199.0,7600.0,8.04,1200.0,2100.0,1.605836e+15,NaN,103200.0,3234867,93012000.0,6710.3909


In [121]:
SH = SH[SH['ID'] == 1603958]
SH['safeBid1p'] = np.where(SH['bid1p'] == 0, SH['ask1p'], SH['bid1p'])
SH['safeAsk1p'] = np.where(SH['ask1p'] == 0, SH['bid1p'], SH['ask1p'])
SH['adjMid'] = (SH['safeBid1p']*SH['ask1q'] + SH['safeAsk1p']*SH['bid1q'])/(SH['bid1q'] + SH['ask1q'])
SH

,ID,ask1p,ask1q,ask5q,bid1p,bid1q,bid5q,clockAtArrival,close,cum_volume,sequenceNo,time,indexClose,safeBid1p,safeAsk1p,adjMid,session,sessionStartCLA,relativeClock,trainFlag,index,sessionStartIx,F30s_ix,F90s_ix,F300s_ix,adjMid_F30s,adjMid_F90s,adjMid_F300s,indexClose_F30s,indexClose_F90s,indexClose_F300s
355,1603958,8.06,11599.0,3100.0,8.05,8600.0,2100.0,1.605836e+15,NaN,23700.0,1999915,93000000.0,6713.6481,8.05,8.06,8.054258,0,1.605836e+15,0.000000e+00,0,355,355,19,60,203,8.035849,8.074545,8.165752,6707.0773,6709.8323,6720.0956
356,1603958,8.06,8399.0,3100.0,8.05,8600.0,2100.0,1.605836e+15,NaN,26900.0,2039987,93000000.0,6713.6481,8.05,8.06,8.055059,0,1.605836e+15,2.849900e+05,0,356,355,19,60,203,8.035849,8.074545,8.165752,6707.0773,6709.8323,6720.0956
357,1603958,8.12,27700.0,7100.0,8.06,6701.0,2100.0,1.605836e+15,NaN,81699.0,2436140,93002000.0,6713.6481,8.06,8.12,8.071687,0,1.605836e+15,2.997814e+06,0,357,355,21,62,205,NaN,8.075263,8.149798,6703.8850,6709.8323,6720.0956
358,1603958,8.12,27700.0,7100.0,8.06,5501.0,2100.0,1.605836e+15,NaN,82899.0,2489528,93003000.0,6713.6481,8.06,8.12,8.069941,0,1.605836e+15,3.388050e+06,0,358,355,21,62,205,NaN,8.075263,8.149798,6703.8850,6709.8323,6720.0956
360,1603958,8.11,6000.0,5000.0,8.06,5301.0,1500.0,1.605836e+15,NaN,91799.0,2877365,93006000.0,6712.4320,8.06,8.11,8.083454,0,1.605836e+15,6.062890e+06,0,360,355,22,64,207,8.049775,8.075263,8.159596,6703.8850,6709.8696,6719.8930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13195,1603958,7.52,14500.0,9100.0,7.51,2100.0,7600.0,1.605856e+15,NaN,15057399.0,131290732,151109000.0,6766.0605,7.51,7.52,7.511265,1,1.605848e+15,7.869944e+09,1,13195,6751,4636,-1,-1,7.499917,NaN,NaN,6764.0037,NaN,NaN
13197,1603958,7.52,14500.0,9100.0,7.51,2100.0,7600.0,1.605856e+15,NaN,15057399.0,131292157,151126000.0,6766.0605,7.51,7.52,7.511265,1,1.605848e+15,7.886482e+09,1,13197,6751,-1,-1,-1,NaN,NaN,NaN,NaN,NaN,NaN
13200,1603958,7.52,14500.0,9100.0,7.51,2100.0,7600.0,1.605856e+15,NaN,15057399.0,131294790,151156000.0,6766.0605,7.51,7.52,7.511265,1,1.605848e+15,7.916487e+09,1,13200,6751,4638,-1,-1,7.488953,NaN,NaN,6764.0037,NaN,NaN
13201,1603958,7.52,14500.0,9100.0,7.51,2100.0,7600.0,1.605856e+15,NaN,15057399.0,131300621,151209000.0,6766.0605,7.51,7.52,7.511265,1,1.605848e+15,7.929946e+09,1,13201,6751,-1,-1,-1,NaN,NaN,NaN,NaN,NaN,NaN


In [122]:
SH['session'] = np.where(SH['time'] >= 130000000, 1, 0)
def findTmValue(clockLs, tm, method='L', buffer=0):
    maxIx = len(clockLs)
    orignIx = np.arange(maxIx)
    if method == 'F':
        ix = np.searchsorted(clockLs, clockLs+(tm-buffer))
        ## if target future index is next tick, mask
        mask = (orignIx == (ix - 1))|(orignIx == ix)|(ix == maxIx)
    elif method == 'L':
        ## if target future index is last tick, mask
        ix = np.searchsorted(clockLs, clockLs-(tm-buffer))
        ix = ix - 1
        ix[ix<0] = 0
        ## !!!ATTENTION: model3 change
        mask = (orignIx == ix) | ((clockLs-(tm-buffer)).values < clockLs.values[0])
    ix[mask] = -1
    return ix

SH = SH.reset_index(drop=True)
groupAllData = SH.groupby(['ID', 'session'])
SH['sessionStartCLA'] = groupAllData['clockAtArrival'].transform('min')
SH['relativeClock'] = SH['clockAtArrival'] - SH['sessionStartCLA']
SH['trainFlag'] = np.where(SH['relativeClock'] > 179.5*1e6, 1, 0)
SH['index'] = SH.index.values
SH['sessionStartIx'] = groupAllData['index'].transform('min')
for tm in [30, 90, 300]:
    tmCol = 'F{}s_ix'.format(tm)
    SH[tmCol] = groupAllData['relativeClock'].transform(lambda x: findTmValue(x, tm*1e6, 'F', 5*1e5)).astype(int)
nearLimit = ((SH.ask5q.values == 0) | (SH.bid5q.values == 0))

for tm in [30, 90, 300]:
    tmIx = SH['F{}s_ix'.format(tm)].values + SH['sessionStartIx'].values
    adjMid_tm = SH['adjMid'].values[tmIx]
    adjMid_tm[SH['F{}s_ix'.format(tm)].values == -1] = np.nan
    SH['adjMid_F{}s'.format(tm)] = adjMid_tm

for tm in [30, 90, 300]:
    tmIx = SH['F{}s_ix'.format(tm)].values + SH['sessionStartIx'].values
    adjMid_tm = SH['indexClose'].values[tmIx]
    adjMid_tm[SH['F{}s_ix'.format(tm)].values == -1] = np.nan
    SH['indexClose_F{}s'.format(tm)] = adjMid_tm

In [78]:
pd.set_option('max_rows', 200)
SH[SH['time'] >= 130000000].head(100)

,ID,ask1p,ask1q,ask5q,bid1p,bid1q,bid5q,clockAtArrival,close,cum_volume,sequenceNo,time,indexClose,safeBid1p,safeAsk1p,adjMid,session,sessionStartCLA,relativeClock,trainFlag,index,sessionStartIx,F30s_ix,F90s_ix,F300s_ix,adjMid_F30s,adjMid_F90s,adjMid_F300s,indexClose_F30s,indexClose_F90s,indexClose_F300s
6751,1603958,8.22,1500.0,12000.0,8.21,3000.0,5500.0,1.605848e+15,NaN,5917899.0,78466720,130000000.0,6733.4999,8.21,8.22,8.216667,1,1.605848e+15,0.0,0,6751,6751,21,62,207,8.233105,NaN,7.774800,6737.8264,6740.8987,6738.9305
6752,1603958,8.33,2900.0,300.0,8.32,460000.0,3400.0,1.605848e+15,NaN,7156999.0,78659133,130001000.0,6733.4999,8.32,8.33,8.329937,1,1.605848e+15,1409304.0,0,6752,6751,22,63,208,NaN,8.230204,7.777200,6738.2703,6740.8987,6738.9305
6753,1603958,8.32,219100.0,37300.0,8.31,465600.0,3400.0,1.605848e+15,NaN,8219599.0,78891132,130003000.0,6733.4999,8.31,8.32,8.316800,1,1.605848e+15,3245346.0,0,6753,6751,23,64,209,8.233105,8.230204,NaN,6738.2703,6740.8987,6738.8764
6755,1603958,8.32,218400.0,37300.0,8.31,441500.0,3400.0,1.605848e+15,NaN,8245399.0,78955697,130004000.0,6733.6911,8.31,8.32,8.316690,1,1.605848e+15,4442439.0,0,6755,6751,24,65,210,8.273333,8.230204,7.780303,6738.2703,6740.8987,6738.8764
6756,1603958,8.32,218300.0,37300.0,8.31,394900.0,3400.0,1.605848e+15,NaN,8292699.0,79014601,130006000.0,6733.6911,8.31,8.32,8.316440,1,1.605848e+15,6124721.0,0,6756,6751,25,66,211,8.273333,8.230139,NaN,6738.2703,6740.8987,6738.8764
6757,1603958,8.32,216400.0,37300.0,8.31,334900.0,3400.0,1.605848e+15,NaN,8354899.0,79049642,130007000.0,6733.6911,8.31,8.32,8.316075,1,1.605848e+15,7405842.0,0,6757,6751,26,66,212,8.265000,8.230139,7.780385,6738.2703,6740.8987,6738.8764
6758,1603958,8.32,216400.0,37300.0,8.31,334900.0,3400.0,1.605848e+15,NaN,8354899.0,79087037,130009000.0,6733.6911,8.31,8.32,8.316075,1,1.605848e+15,9040337.0,0,6758,6751,27,68,213,NaN,8.230139,7.780016,6738.7906,6740.7537,6738.8764
6760,1603958,8.32,216400.0,37300.0,8.31,254900.0,3400.0,1.605848e+15,NaN,8434899.0,79112409,130010000.0,6735.4826,8.31,8.32,8.315408,1,1.605848e+15,10365304.0,0,6760,6751,28,68,214,8.265000,8.230139,NaN,6738.7906,6740.7537,6738.9513
6761,1603958,8.32,215700.0,37300.0,8.31,254900.0,3400.0,1.605848e+15,NaN,8435599.0,79128633,130012000.0,6735.4826,8.31,8.32,8.315416,1,1.605848e+15,11999227.0,0,6761,6751,29,70,215,8.265455,8.220244,7.780016,6738.7906,6740.7537,6738.9513
6762,1603958,8.32,212200.0,37300.0,8.31,164900.0,3400.0,1.605848e+15,NaN,8529099.0,79149536,130013000.0,6735.4826,8.31,8.32,8.314373,1,1.605848e+15,13348515.0,0,6762,6751,30,70,216,8.264706,8.220244,7.791031,6738.7906,6740.7537,6738.9513


In [131]:
SH[SH['cum_volume'] >= 8804899]['clockAtArrival'].iloc[0]

1605848490399767.0

In [137]:
SH[(SH['clockAtArrival'] >= 1605848490823265) & (SH['clockAtArrival']<= 1605848520400990)]

,ID,ask1p,ask1q,ask5q,bid1p,bid1q,bid5q,clockAtArrival,close,cum_volume,sequenceNo,time,indexClose,safeBid1p,safeAsk1p,adjMid,session,sessionStartCLA,relativeClock,trainFlag,index,sessionStartIx,F30s_ix,F90s_ix,F300s_ix,adjMid_F30s,adjMid_F90s,adjMid_F300s,indexClose_F30s,indexClose_F90s,indexClose_F300s
4411,1603958,8.16,24900.0,9900.0,8.15,33100.0,4000.0,1.605848e+15,NaN,8810199.0,79662438,130122000.0,6740.1933,8.15,8.16,8.155707,1,1.605848e+15,82342977.0,0,4411,4355,76,118,262,8.075060,7.869204,7.634369,6738.6721,6737.4220,6738.4070
4412,1603958,8.16,24900.0,9900.0,8.15,33100.0,4000.0,1.605848e+15,NaN,8810199.0,79674173,130124000.0,6740.1345,8.15,8.16,8.155707,1,1.605848e+15,84598730.0,0,4412,4355,78,119,265,8.073022,7.872483,7.580476,6738.7555,6737.6076,6738.6352
4413,1603958,8.16,24700.0,9900.0,8.15,33100.0,4000.0,1.605848e+15,NaN,8810399.0,79677481,130124000.0,6740.1345,8.15,8.16,8.155727,1,1.605848e+15,84907117.0,0,4413,4355,78,119,265,8.073022,7.872483,7.580476,6738.7555,6737.6076,6738.6352
4414,1603958,8.16,24300.0,9900.0,8.15,33100.0,4000.0,1.605848e+15,NaN,8810799.0,79681584,130125000.0,6740.1345,8.15,8.16,8.155767,1,1.605848e+15,85344050.0,0,4414,4355,79,120,265,8.054651,7.869117,7.580476,6738.7555,6737.6076,6738.6352
4415,1603958,8.16,24200.0,9900.0,8.15,33100.0,4000.0,1.605848e+15,NaN,8810899.0,79695607,130127000.0,6740.1345,8.15,8.16,8.155777,1,1.605848e+15,87904227.0,0,4415,4355,81,121,267,8.061846,7.881613,7.580761,6738.7555,6737.6076,6738.6352
4416,1603958,8.16,23300.0,9900.0,8.15,33100.0,4000.0,1.605848e+15,NaN,8811799.0,79699161,130128000.0,6740.1345,8.15,8.16,8.155869,1,1.605848e+15,88339261.0,0,4416,4355,81,122,267,8.061846,7.881630,7.580761,6738.7555,6737.6076,6738.6352
4417,1603958,8.15,17300.0,39000.0,8.14,8400.0,6100.0,1.605848e+15,NaN,8845599.0,79713386,130130000.0,6739.9959,8.14,8.15,8.143268,1,1.605848e+15,90897612.0,0,4417,4355,83,123,269,8.070937,7.733103,7.585578,6738.7220,6737.8744,6738.4298
4418,1603958,8.15,17300.0,39000.0,8.14,6400.0,6100.0,1.605848e+15,NaN,8847899.0,79717168,130131000.0,6739.9959,8.14,8.15,8.142700,1,1.605848e+15,91346787.0,0,4418,4355,83,124,269,8.070937,7.733103,7.585578,6738.7220,6737.8744,6738.4298
4419,1603958,8.15,14900.0,39000.0,8.14,6400.0,6100.0,1.605849e+15,NaN,8850299.0,79731625,130133000.0,6739.9959,8.14,8.15,8.143005,1,1.605848e+15,93910149.0,0,4419,4355,85,125,271,8.066053,7.886269,7.585823,6738.7220,6737.8744,6738.6714
4420,1603958,8.15,14600.0,39000.0,8.14,6300.0,6100.0,1.605849e+15,NaN,8850699.0,79735519,130134000.0,6739.6535,8.14,8.15,8.143014,1,1.605848e+15,94351158.0,0,4420,4355,85,126,271,8.066053,7.886269,7.585823,6738.7220,6737.6018,6738.6714


In [62]:
SH['clockAtArrival'].iloc[1] - SH['clockAtArrival'].iloc[0]

202056.0

In [37]:
# 20201120 1603958 8804899 975601
orderLog[(orderLog['secid'] == 1603958) & (orderLog['internalId'] == 562) & (orderLog['accCode'] == 975601) & (orderLog['absOrderSize'] == 700)]

,Unnamed: 0,date,clockAtArrival,caamd,colo,accCode,secid,vai,sdd,orderDirection,absOrderSize,updateType,tradePrice,absFilledThisUpdate,absOrderSizeCumFilled,tradeId,orderSysId,aaa,internalId,ars,mse,mrsb300,mfe,mra100,session,mrmum,hee,mt,sequenceNo,mrsb90,mrss90,ms,threadId,mta,mrstauc,orderPrice,finalState,mrrlma,l4tr,mrstaum,mrm25,clock,cancellationPending,mrstaat,mrss300,underlyingIndex,ApplSeqNum,cfe,mrm,mrb100,orderId
247808,247808,20201120,1.605848e+15,1.605848e+15,zs_96_06,975601,1603958.0,8804899.0,130121000.0,1.0,700.0,0.0,-1.00,0.0,0.0,NaN,NaN,0.0038,562.0,1.0,13.0,0.003912,0.0,816.0,0,-1.0,0.001332,0.0,135953459.0,0.002997,-0.004339,13:01:22.052162,21121.0,0.004159,10000.0,8.16,0.0,0.0,0.0,1.0,-1.0,1.605848e+15,0.0,11000.0,-0.005118,852,0.0,16182276.16,0.004159,815.0,-1.000000e+00
247809,247809,20201120,1.605848e+15,1.605848e+15,zs_96_06,975601,1603958.0,-1.0,-1.0,1.0,700.0,2.0,8.16,0.0,0.0,NaN,7.214336671251187e+17,-1,562.0,-1.0,0.0,0.003912,0.0,816.0,0,-1.0,-1.000000,-1.0,135953459.0,0.002997,-0.004339,13:01:22.273162,21228.0,-1.000000,10000.0,8.16,0.0,0.0,0.0,1.0,-1.0,1.605848e+15,0.0,11000.0,-0.005118,852,0.0,16182276.16,0.004159,815.0,7.214337e+17
247810,247810,20201120,1.605848e+15,1.605848e+15,zs_96_06,975601,1603958.0,-1.0,-1.0,1.0,700.0,4.0,8.16,700.0,700.0,47351,NaN,-1,562.0,-1.0,0.0,0.003912,0.0,816.0,0,-1.0,-1.000000,-1.0,135953459.0,0.002997,-0.004339,13:01:22.420162,21228.0,-1.000000,10000.0,8.16,1.0,0.0,0.0,1.0,-1.0,1.605848e+15,0.0,11000.0,-0.005118,852,0.0,16182276.16,0.004159,815.0,7.214337e+17


In [101]:
re = pd.read_pickle(r'L:\orderLog\ret\orderLogWithRet_20201120.pkl')
re[(re['secid'] == 1603958) & (re['accCode'] == 975601) & (re['vai'] == 8804899)]

,Unnamed: 0,date,clockAtArrival,caamd,colo,accCode,secid,vai,sdd,orderDirection,absOrderSize,updateType,tradePrice,absFilledThisUpdate,absOrderSizeCumFilled,tradeId,orderSysId,aaa,internalId,ars,mse,mrsb300,mfe,mra100,session,mrmum,hee,mt,sequenceNo,mrsb90,mrss90,ms,threadId,mta,mrstauc,orderPrice,finalState,mrrlma,l4tr,mrstaum,mrm25,clock,cancellationPending,mrstaat,mrss300,underlyingIndex,ApplSeqNum,cfe,mrm,mrb100,orderId,test,t1,order,firstUpdateType,insertNum,innerSeq,firstClock,indexCat,beta_60,mdStartClock,mdClockAtArrival,adjMid_F30s,adjMid_F90s,adjMid_F300s,indexClose,indexClose_F30s,indexClose_F90s,indexClose_F300s
271960,247808,20201120,1.605848e+15,1.605848e+15,zs_96_06,975601,1603958.0,8804899.0,130121000.0,1.0,700.0,0.0,-1.00,0.0,0.0,NaN,NaN,0.0038,562.0,1.0,13.0,0.003912,0.0,816.0,0,-1.0,0.001332,0.0,135953459.0,0.002997,-0.004339,13:01:22.052162,21121.0,0.004159,10000.0,8.16,0.0,0.0,0.0,1.0,-1.0,1.605848e+15,0.0,11000.0,-0.005118,852,0.0,16182276.16,0.004159,815.0,-1.000000e+00,60768,1.0,89817,0.0,1,247808,1.605848e+15,1000852,0.877165,1.605848e+15,1.605848e+15,8.07506,7.869204,7.634369,6740.1933,6738.6721,6737.422,6738.407
271961,247809,20201120,1.605848e+15,1.605848e+15,zs_96_06,975601,1603958.0,8804899.0,-1.0,1.0,700.0,2.0,8.16,0.0,0.0,NaN,7.214336671251187e+17,-1,562.0,-1.0,0.0,0.003912,0.0,816.0,0,-1.0,-1.000000,-1.0,135953459.0,0.002997,-0.004339,13:01:22.273162,21228.0,-1.000000,10000.0,8.16,0.0,0.0,0.0,1.0,-1.0,1.605848e+15,0.0,11000.0,-0.005118,852,0.0,16182276.16,0.004159,815.0,7.214337e+17,60768,1.0,89817,0.0,1,247809,1.605848e+15,1000852,0.877165,1.605848e+15,1.605848e+15,8.07506,7.869204,7.634369,6740.1933,6738.6721,6737.422,6738.407
271962,247810,20201120,1.605848e+15,1.605848e+15,zs_96_06,975601,1603958.0,8804899.0,-1.0,1.0,700.0,4.0,8.16,700.0,700.0,47351,NaN,-1,562.0,-1.0,0.0,0.003912,0.0,816.0,0,-1.0,-1.000000,-1.0,135953459.0,0.002997,-0.004339,13:01:22.420162,21228.0,-1.000000,10000.0,8.16,1.0,0.0,0.0,1.0,-1.0,1.605848e+15,0.0,11000.0,-0.005118,852,0.0,16182276.16,0.004159,815.0,7.214337e+17,60768,1.0,89817,0.0,1,247810,1.605848e+15,1000852,0.877165,1.605848e+15,1.605848e+15,8.07506,7.869204,7.634369,6740.1933,6738.6721,6737.422,6738.407


In [103]:
read_beta_daily(db1, 'mktbeta', 20201120, 20201120, skey=[1603958])

,skey,name,date,time,beta_10d_IF,beta_60d_IF,beta_10d_IC,beta_60d_IC,beta_10d_CSI1000,beta_60d_CSI1000,alpha_10d_IF,alpha_60d_IF,alpha_10d_IC,alpha_60d_IC,alpha_10d_CSI1000,alpha_60d_CSI1000
0,1603958,哈森股份,20201120,213000000000,2.0,1.005074,1.019126,0.991116,0.655005,0.877165,0.008602,-0.00045,0.010269,0.000894,0.011172,0.001258


In [141]:
re[(re['secid'] == 1603958)  & (re['vai'] == 8804899) & (re['accCode'] == 975601)]

,Unnamed: 0,date,clockAtArrival,caamd,colo,accCode,secid,vai,sdd,orderDirection,absOrderSize,updateType,tradePrice,absFilledThisUpdate,absOrderSizeCumFilled,tradeId,orderSysId,aaa,internalId,ars,mse,mrsb300,mfe,mra100,session,mrmum,hee,mt,sequenceNo,mrsb90,mrss90,ms,threadId,mta,mrstauc,orderPrice,finalState,mrrlma,l4tr,mrstaum,mrm25,clock,cancellationPending,mrstaat,mrss300,underlyingIndex,ApplSeqNum,cfe,mrm,mrb100,orderId,test,t1,order,firstUpdateType,insertNum,innerSeq,firstClock,indexCat,beta_60,mdStartClock,mdClockAtArrival,adjMid_F30s,adjMid_F90s,adjMid_F300s,indexClose,indexClose_F30s,indexClose_F90s,indexClose_F300s
271960,247808,20201120,1.605848e+15,1.605848e+15,zs_96_06,975601,1603958.0,8804899.0,130121000.0,1.0,700.0,0.0,-1.00,0.0,0.0,NaN,NaN,0.0038,562.0,1.0,13.0,0.003912,0.0,816.0,0,-1.0,0.001332,0.0,135953459.0,0.002997,-0.004339,13:01:22.052162,21121.0,0.004159,10000.0,8.16,0.0,0.0,0.0,1.0,-1.0,1.605848e+15,0.0,11000.0,-0.005118,852,0.0,16182276.16,0.004159,815.0,-1.000000e+00,60768,1.0,89817,0.0,1,247808,1.605848e+15,1000852,0.877165,1.605848e+15,1.605848e+15,8.07506,7.869204,7.634369,6740.1933,6738.6721,6737.422,6738.407
271961,247809,20201120,1.605848e+15,1.605848e+15,zs_96_06,975601,1603958.0,8804899.0,-1.0,1.0,700.0,2.0,8.16,0.0,0.0,NaN,7.214336671251187e+17,-1,562.0,-1.0,0.0,0.003912,0.0,816.0,0,-1.0,-1.000000,-1.0,135953459.0,0.002997,-0.004339,13:01:22.273162,21228.0,-1.000000,10000.0,8.16,0.0,0.0,0.0,1.0,-1.0,1.605848e+15,0.0,11000.0,-0.005118,852,0.0,16182276.16,0.004159,815.0,7.214337e+17,60768,1.0,89817,0.0,1,247809,1.605848e+15,1000852,0.877165,1.605848e+15,1.605848e+15,8.07506,7.869204,7.634369,6740.1933,6738.6721,6737.422,6738.407
271962,247810,20201120,1.605848e+15,1.605848e+15,zs_96_06,975601,1603958.0,8804899.0,-1.0,1.0,700.0,4.0,8.16,700.0,700.0,47351,NaN,-1,562.0,-1.0,0.0,0.003912,0.0,816.0,0,-1.0,-1.000000,-1.0,135953459.0,0.002997,-0.004339,13:01:22.420162,21228.0,-1.000000,10000.0,8.16,1.0,0.0,0.0,1.0,-1.0,1.605848e+15,0.0,11000.0,-0.005118,852,0.0,16182276.16,0.004159,815.0,7.214337e+17,60768,1.0,89817,0.0,1,247810,1.605848e+15,1000852,0.877165,1.605848e+15,1.605848e+15,8.07506,7.869204,7.634369,6740.1933,6738.6721,6737.422,6738.407
